<a href="https://colab.research.google.com/github/TobiePham/Wood_Classification/blob/master/BE_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyngrok
!ngrok authtoken 2JOsPqiBNJ0lpyNRIWUImBUFr7d_7ncgCPc8KdLDts77ZZA83

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 29.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=c480e25df4ab7b82f58fc2bb69a9aef7015c76ba2b038b27d946347aa487ae23
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
# from tensorflow.keras.utils import load_img,image_to_array
import pickle
import cv2
import base64
import numpy as np

In [ ]:
model= DenseNet121(input_shape= (224, 224, 3),include_top = False,weights = 'imagenet', pooling='avg')
ss=pickle.load(open("/content/drive/MyDrive/Project/BE/ss.pickle","rb"))
gmm_clf=pickle.load(open("/content/drive/MyDrive/Project/BE/gmm_clf.pickle","rb"))
pca=pickle.load(open("/content/drive/MyDrive/Project/BE/pca.pickle","rb"))
isotonic_regressor=pickle.load(open("/content/drive/MyDrive/Project/BE/isotonic_regressor.pickle","rb"))

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)  
#load model

def chuyen_base64_sang_anh(anh_base64):
    try:
        anh_base64 = np.frombuffer(base64.b64decode(anh_base64), dtype=np.uint8)
        anh_base64 = cv2.imdecode(anh_base64, cv2.IMREAD_ANYCOLOR)
    except:
        return None
    return anh_base64

anh=""

@app.route('/detectWoodImage', methods=['POST'] )
def nhandienkhuonmat_process():
    image= request.args.get('image')
    anh=image
    image= chuyen_base64_sang_anh(image)
    image = cv2.resize(image,(224,224))
    image = np.reshape(image,[1,224,224,3])
    print(image.shape)

    output = preprocess_input(image)

    feature_extract= model.predict(output)

    feature_extract = ss.transform(feature_extract)

    feature_extract = pca.transform(feature_extract)
    log_probs_test = gmm_clf.score_samples(feature_extract)
    test_probabilities = isotonic_regressor.predict(log_probs_test)
    test_predictions = [1 if prob >= 0.5 else 0 for prob in test_probabilities]
    # Trả về
    print(test_predictions)
    if(test_predictions[0]==0):
      return "0"
    else:
      return "1"

# Start Backend
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b4d3-34-86-138-115.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
(1, 224, 224, 3)
1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [29/Dec/2022 18:54:02] "POST /detectWoodImage?image=%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAIQAABtbnRyUkdC%0AIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAA%0AAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlk%0AZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAA%0AAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAA%0AAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA%0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAA%0AAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA%2BEAAC2z3Bh%0AcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADT%0ALW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAw%0AADEANv%2FbAEMAEAsMDgwKEA4NDhIREBMYKBoYFhYYMSMlHSg6Mz08OTM4N0BIXE5ARFdFNzhQbVFX%0AX2JnaGc%2BTXF5cGR4XGVnY%2F%2FbAEMBERISGBU

[0]
(1, 224, 224, 3)
1/1 [==============================] - 0s 129ms/step


...

[1]
(1, 224, 224, 3)
1/1 [==============================] - 0s 129ms/step
[1]


INFO:werkzeug:127.0.0.1 - - [29/Dec/2022 18:56:02] "POST /detectWoodImage?image=%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAIQAABtbnRyUkdC%0AIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAA%0AAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlk%0AZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAA%0AAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAA%0AAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA%0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAA%0AAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA%2BEAAC2z3Bh%0AcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADT%0ALW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAw%0AADEANv%2FbAEMAEAsMDgwKEA4NDhIREBMYKBoYFhYYMSMlHSg6Mz08OTM4N0BIXE5ARFdFNzhQbVFX%0AX2JnaGc%2BTXF5cGR4XGVnY%2F%2FbAEMBERISGBU

(1, 224, 224, 3)
1/1 [==============================] - 0s 123ms/step


INFO:werkzeug:127.0.0.1 - - [29/Dec/2022 18:56:35] "POST /detectWoodImage?image=%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAIQAABtbnRyUkdC%0AIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAA%0AAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlk%0AZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAA%0AAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAA%0AAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA%0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAA%0AAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA%2BEAAC2z3Bh%0AcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADT%0ALW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAw%0AADEANv%2FbAEMAEAsMDgwKEA4NDhIREBMYKBoYFhYYMSMlHSg6Mz08OTM4N0BIXE5ARFdFNzhQbVFX%0AX2JnaGc%2BTXF5cGR4XGVnY%2F%2FbAEMBERISGBU

[0]


In [ ]:


# with open("/content/image 1.png", "rb") as img_file:
#     b64_string = base64.b64encode(img_file.read())
# print(b64_string)


# print(chuyen_base64_sang_anh(b64_string))

# from io import BytesIO
# from PIL import Image
# def chuyen_base64_sang_anh(anh_base64):
#   try:
#     anh_base64 = base64.urlsafe_b64decode(anh_base64)
#     print(anh_base64)
#     anh_base64 = np.frombuffer(anh_base64,np.uint8)

#     anh_base64 = cv2.imdecode(anh_base64,cv2.IMREAD_COLOR)
#   except Exception as e: print(e)
#   return anh_base64

img = cv2.imread('/content/image 1.png')
img = cv2.resize(img,(224,224))
img = np.reshape(img,[1,224,224,3])

output = preprocess_input(img)

feature_extract= model.predict(output)

feature_extract = ss.transform(feature_extract)

feature_extract = pca.transform(feature_extract)
log_probs_test = gmm_clf.score_samples(feature_extract)
test_probabilities = isotonic_regressor.predict(log_probs_test)
test_predictions = [1 if prob >= 0.5 else 0 for prob in test_probabilities]
print(test_predictions)

1/1 [==============================] - 3s 3s/step
[1]
